In [ ]:
import os
import random
import pandas as pd
import tensorflow_addons as tfa
import numpy as np
import PIL.Image
import tensorflow as tf

from matplotlib import pyplot as plt
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten

from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import array_to_img, img_to_array

In [ ]:
traing_metadata = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')

def cropping(image, filename=None, path=None, dst=None, central=True, random=False, fraction_low=0.5, fraction_high=0.9,
             random_width=224, random_height=224, copies=3, save_original=True):
    """
    :type copies: int
    :type random_height: int
    :type random_width: int
    :type fraction_high: float
    :type fraction_low: float
    :type random: bool
    :type central: bool
    :type save_original: bool
    :type dst: basestring
    :type path: basestring
    """
    if path is not None:
        images = os.listdir(path)

        for image in images:
            image_path = os.path.join(path, image)
            img = img_to_array(PIL.Image.open(image_path))

            if central:
                for i in range(copies):
                    crop_area = np.round(np.random.uniform(fraction_low, fraction_high), 2)
                    cc_img = tf.image.central_crop(img, central_fraction=crop_area)

                    if dst is not None:
                        array_to_img(cc_img).save(os.path.join(dst, f'cc_img_{i}_{image}'))

                    else:
                        return array_to_img(cc_img)

            elif random:
                for i in range(copies):
                    rc_img = tf.image.random_crop(img, size=[random_width, random_height, 3])

                    if dst is not None:
                        array_to_img(rc_img).save(os.path.join(dst, f'rc_img_{i}_{image}'))

                    else:
                        return array_to_img(rc_img)

            if save_original:
                array_to_img(img).save(os.path.join(dst, image))

    if image:
        if central:
            for i in range(copies):
                crop_area = np.round(np.random.uniform(fraction_low, fraction_high), 2)
                cc_img = tf.image.central_crop(img_to_array(image), central_fraction=crop_area)

                if dst is not None:
                    array_to_img(cc_img).save(os.path.join(dst, f'cc_img_{i}_{filename}'))

                else:
                    return array_to_img(cc_img)

        elif random:
            for i in range(copies):
                rc_img = tf.image.random_crop(img_to_array(image),
                                              size=[random_width, random_height, 3])

                if dst is not None:
                    array_to_img(rc_img).save(os.path.join(dst, f'rc_img_{i}_{filename}'))

                else:
                    return array_to_img(rc_img)

                
def random_changes_to_color_properties(path, dst, delta=None, gamma_transformation=True, change_contrast=True,
                                       factor=None, copies=1, save_original=False):
    """
    :type copies: int
    :type factor: int
    :type change_contrast: bool
    :type gamma_transformation: bool
    :type delta: float
    :type save_original: bool
    :type dst: basestring
    :type path: basestring
    """
    images = os.listdir(path)
    i = 0

    for image in images:
        image_path = os.path.join(path, image)
        img = img_to_array(PIL.Image.open(image_path))

        if delta is None:
            for i in range(copies):
                change_factor = np.round(np.random.uniform(-1, 1), 2)
                bc_img = array_to_img(tf.image.adjust_brightness(img, change_factor))
                hue_img = array_to_img(tf.image.adjust_hue(img, change_factor))
                sat_img = array_to_img(tf.image.adjust_saturation(img, change_factor))

                # save transformed images
                bc_img.save(os.path.join(dst, f'bc_{i}_{image}'))
                hue_img.save(os.path.join(dst, f'hue_{i}_{image}'))
                sat_img.save(os.path.join(dst, f'sat_{i}_{image}'))

        elif isinstance(delta, float):
            for i in range(copies):
                change_factor = np.round(np.random.uniform(-1 * delta, 1 * delta), 2)
                bc_img = array_to_img(tf.image.adjust_brightness(img, change_factor))
                hue_img = array_to_img(tf.image.adjust_hue(img, change_factor))
                sat_img = array_to_img(tf.image.adjust_saturation(img, change_factor))

                # save transformed images
                bc_img.save(os.path.join(dst, f'bc_{i}_{image}'))
                hue_img.save(os.path.join(dst, f'hue_{i}_{image}'))
                sat_img.save(os.path.join(dst, f'sat_{i}_{image}'))

        if factor is None:
            if gamma_transformation:
                for i in range(copies):
                    gamma = np.round(np.random.uniform(1, 5), 2)
                    gamma_img = array_to_img(tf.image.adjust_gamma(img, gamma))
                    gamma_img.save(os.path.join(dst, f'gamma_img_{i}_{image}'))

            if change_contrast:
                for i in range(copies):
                    change_factor = np.round(np.random.uniform(-2, 2), 2)
                    cont_img = array_to_img(tf.image.adjust_contrast(img, change_factor))
                    cont_img.save(os.path.join(dst, f'cont_img_{i}_{image}'))

        elif isinstance(factor, int):
            if gamma_transformation:
                for i in range(copies):
                    gamma = np.round(np.random.uniform(1, factor), 2)
                    gamma_img = array_to_img(tf.image.adjust_gamma(img, gamma))
                    gamma_img.save(os.path.join(dst, f'gamma_img_{i}_{image}'))

            if change_contrast:
                for i in range(copies):
                    factor = np.round(np.random.uniform(-1 * factor, factor), 2)
                    cont_img = array_to_img(tf.image.adjust_contrast(img, factor))
                    cont_img.save(os.path.join(dst, f'cont_img_{i}_{image}'))

        if save_original:
            array_to_img(img).save(os.path.join(dst, image))
          
        i += 1
        print(f"steps: {i}/{len(images)} ", end='\r')

In [ ]:
sample = traing_metadata.sample(n=10)
figure, axes = plt.subplots(nrows=2, ncols=5, figsize=[16,4], dpi=300)
axes = axes.ravel()

for i in range(sample.shape[0]):
    file_name, label = sample.iloc[i]
    src = os.path.join('../input/resizing-high-quality-images/train/', file_name)
    img = PIL.Image.open(src)
    
    axes[i].imshow(img)

In [ ]:
for i in range(traing_metadata.shape[0]):
    file_name, label = traing_metadata.iloc[i]
    
    if not os.path.isdir('train'):
        os.mkdir('train')
        
    if not os.path.isdir(os.path.join('train',label)):
        os.mkdir(os.path.join('train', label))
        
    src = os.path.join('../input/resizing-high-quality-images/train/', file_name)
    dst = os.path.join('train',label)
    img = PIL.Image.open(src)
    
    cc_img = cropping(img, central=True, random=False, fraction_low=0.75, fraction_high=0.9, copies=1, save_original=False)
    cropping(cc_img, file_name, dst=dst, central=False, random=True, random_width=224, random_height=224, copies=5, save_original=False)
    
    print(f"steps: {i + 1}/{traing_metadata.shape[0]} ", end='\r')

In [ ]:
sample = traing_metadata.sample(n=1)
figure, axes = plt.subplots(nrows=1, ncols=5, figsize=[16,4], dpi=300)
axes = axes.ravel()

for i in range(5):
    filename, label = sample.iloc[0]
    src = os.path.join('train', label, f'rc_img_{i}_{filename}')
    img = PIL.Image.open(src)
    
    axes[i].imshow(img)

In [ ]:
for sub_dir in os.listdir('train'):
    src = os.path.join('train', sub_dir)
    dst = os.path.join('train', sub_dir)
    
    random_changes_to_color_properties(src, dst)

In [ ]:
classes = random.choice(os.listdir('train'))

figure, axes = plt.subplots(nrows=2, ncols=5, figsize=[16,4], dpi=300)
axes = axes.ravel()

for i in range(10):
    file_name = random.choice(os.listdir(os.path.join('train', classes)))
    src = os.path.join('train',  classes, file_name)
    img = PIL.Image.open(src)
    print(file_name)
    
    axes[i].imshow(img)

In [ ]:
def inception(x, filters, projection, name=None):
    f_1x1, f_3x3, f_3x3_reduce, f_5x5, f_5x5_reduce = filters
    x1 = Conv2D(filters=f_1x1, kernel_size=(1, 1), strides=(1, 1), activation=relu, padding='same')(x)
    x3_reducer = Conv2D(filters=f_3x3_reduce, kernel_size=(1, 1), strides=(1, 1), activation=relu, padding='same')(x)
    x5_reducer = Conv2D(filters=f_5x5_reduce, kernel_size=(1, 1), strides=(1, 1), activation=relu, padding='same')(x)
    pool = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)

    x3 = Conv2D(filters=f_3x3, kernel_size=(3, 3), strides=(1, 1), activation=relu, padding='same')(x3_reducer)
    x5 = Conv2D(filters=f_5x5, kernel_size=(5, 5), strides=(1, 1), activation=relu, padding='same')(x5_reducer)
    proj = Conv2D(filters=projection, kernel_size=(1, 1), strides=(1, 1), activation=relu, padding='same')(pool)

    x = concatenate([x1, x3, x5, proj], axis=3, name=name)

    return x


def model_builder(shape, classes):
    input_layer = Input(shape=shape)
    x = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), activation=relu, padding='same')(input_layer)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=64, kernel_size=(1, 1), strides=(1, 1), activation=relu, padding='same')(x)
    x = Conv2D(filters=192, kernel_size=(3, 3), strides=(1, 1), activation=relu, padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = inception(x, [64, 128, 96, 32, 16], projection=32, name='inception_3a')
    x = inception(x, [128, 192, 128, 96, 32], projection=64, name='inception_3b')
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = inception(x, [192, 208, 96, 48, 16], projection=64, name='inception_4a')

    aux_1 = AveragePooling2D(pool_size=(5, 5), strides=(3, 3), padding='valid')(x)
    aux_1 = Conv2D(filters=128, kernel_size=(1, 1), strides=(1, 1), activation=relu, padding='valid')(aux_1)
    aux_1 = Dense(units=1024, activation=relu)(aux_1)
    aux_1 = Dropout(rate=0.7)(aux_1)
    aux_1 = Flatten()(aux_1)
    aux_out1 = Dense(units=classes, activation=softmax, name='aux_out1')(aux_1)

    x = inception(x, [160, 224, 112, 64, 24], projection=64, name='inception_4b')
    x = inception(x, [128, 256, 128, 64, 24], projection=64, name='inception_4c')
    x = inception(x, [112, 288, 144, 64, 32], projection=64, name='inception_4d')
    x = inception(x, [256, 320, 160, 128, 32], projection=128, name='inception_4e')

    aux_2 = AveragePooling2D(pool_size=(5, 5), strides=(3, 3), padding='valid')(x)
    aux_2 = Conv2D(filters=128, kernel_size=(1, 1), strides=(1, 1), activation=relu, padding='valid')(aux_2)
    aux_2 = Dense(units=1024, activation=relu)(aux_2)
    aux_2 = Dropout(rate=0.7)(aux_2)
    aux_2 = Flatten()(aux_2)
    aux_out2 = Dense(units=classes, activation=softmax, name='aux_out2')(aux_2)

    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = inception(x, [256, 320, 160, 128, 32], projection=128, name='inception_5a')
    x = inception(x, [384, 384, 192, 128, 48], projection=128, name='inception_5b')
    x = AveragePooling2D(pool_size=(7, 7), strides=(1, 1))(x)
    x = Dropout(rate=0.4)(x)
    x = Flatten()(x)
    output_layer = Dense(units=classes, activation=softmax, name='main_out')(x)

    model = Model(input_layer, [output_layer, aux_out1, aux_out2])
    model.compile(optimizer=Adam(), loss=categorical_crossentropy,
                  loss_weights={'main_out': 1, 'aux_out1': 0.3, 'aux_out2': 0.3},
                  metrics=['accuracy', tfa.metrics.F1Score(num_classes=classes, threshold=0.5)])
    model.summary()

    return model

In [ ]:
train_generator = ImageDataGenerator(rescale=1 / 255.,
                                     validation_split=0.3)

train_set = train_generator.flow_from_directory('train',
                                                target_size=(224, 224),
                                                batch_size=32,
                                                subset='training')
val_set = train_generator.flow_from_directory('train',
                                              target_size=(224, 224),
                                              batch_size=32,
                                              subset='validation')

In [ ]:
model = model_builder((224, 224, 3), 12)

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.1,
                              patience=10)

In [ ]:
history = model.fit(x=train_set,
                    validation_data=val_set,
                    epochs=100,
                    steps_per_epoch=6114,
                    validation_steps=2620,
                    callbacks=[early_stop, reduce_lr])

In [ ]:
temp = pd.DataFrame(history.history)
temp.to_pickle('inception_aux_random_crop_history.pkl')

In [ ]:
model.save('inception_aux_random_crop.hdf5')

### VGGNet

In [ ]:
# weight_decay = 5e-4

# def vgg_net16_d(input_shape=(224, 224, 3), classes=None):
#     # input layer
#     input_layer = Input(shape=input_shape, name='input_')

#     # first conv block
#     x = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same',
#                activation=relu)(input_layer)
#     x = Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same',
#                activation=relu)(x)
#     x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x)

#     # second conv block
#     x = Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same',
#                activation=relu)(x)
#     x = Conv2D(filters=128, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same',
#                activation=relu)(x)
#     x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x)

#     # third conv block
#     x = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same',
#                activation=relu)(x)
#     x = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same',
#                activation=relu)(x)
#     x = Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same',
#                activation=relu)(x)
#     x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x)

#     # fourth conv block
#     x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same',
#                activation=relu)(x)
#     x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same',
#                activation=relu)(x)
#     x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same',
#                activation=relu)(x)
#     x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x)

#     # fifth conv block
#     x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same',
#                activation=relu)(x)
#     x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same',
#                activation=relu)(x)
#     x = Conv2D(filters=512, kernel_size=(3, 3), strides=(1, 1), kernel_regularizer=l2(weight_decay), padding='same',
#                activation=relu)(x)
#     x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(x)

#     # classifier
#     x = Flatten()(x)
#     x = Dense(units=512, activation=relu)(x)
#     x = Dropout(rate=0.5)(x)
#     x = Dense(units=512, activation=relu)(x)
#     x = Dropout(rate=0.5)(x)
#     x = Dense(units=classes, activation=softmax)(x)

#     model = Model(input_layer, x)
#     model.compile(optimizer=Adam(), loss=categorical_crossentropy, metrics=['accuracy',tfa.metrics.F1Score(num_classes=classes, threshold=0.5)])
#     model.summary()

#     return model

In [ ]:
# vgg_model = vgg_net16_d((224, 224, 3), 12)

In [ ]:
# vgg_history = vgg_model.fit(x=train_set,
#                     validation_data=val_set,
#                     epochs=100,
#                     steps_per_epoch=6114,
#                     validation_steps=2620,
#                     callbacks=[early_stop, reduce_lr])

In [ ]:
# temp = pd.DataFrame(vgg_history.history)
# temp.to_pickle('vgg_random_crop_history.pkl')

In [ ]:
# model.save('vgg_random_crop.hdf5')

In [ ]:
# model.save('vgg_random_crop.hdf5')